In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import gensim
import nltk
from scipy.spatial import distance
import re
from random import shuffle
import string
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn import utils
from sklearn.model_selection import train_test_split
import ast
import pickle
import pprint
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cocoly\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cocoly\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### Model


In [46]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
df_final = pd.read_csv('movies.csv', index_col=0) 
df_final['Description'] = df_final['Description'].apply(lambda x: str(x).strip())

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("turkish"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    new_string = text.translate(str.maketrans('', '', string.punctuation))
    return new_string

# Description only
df_final['cleaned'] = df_final['Description']
# Stars + Directors + Description
df_final = df_final.assign(combine=[(f'''{str(df_final["Title"][i]).strip("[]").replace("'", "").replace(",",".")} ,
{str(df_final["Genres"][i]).strip("[]").replace("'", "").replace(",",".")} ,
{str(df_final["Stars"][i]).strip("[]").replace("'", "").replace(",",".")} ,
{str(df_final["Director"][i]).strip("[]").replace("'", "").replace(",",".")},
{str(df_final["Genres"][i]).strip("[]").replace("'", "").replace(",",".")},
{str(df_final["Genres"][i]).strip("[]").replace("'", "").replace(",",".")}''').lower() for i in range(len(df_final["Title"]))])
# Title + Description
df_final = df_final.assign(temp_f=[(f'''{str(df_final["Stars"][i]).strip("[]").replace("'", "").replace(",",".")}, {str(df_final["Director"][i]).strip("[]").replace("'", "").replace(",",".")}, {df_final['Description']}''').lower() for i in range(len(df_final["Title"]))])

df_final['cleaned'] = df_final.cleaned.apply(func = make_lower_case)
df_final['cleaned'] = df_final.cleaned.apply(func = remove_stop_words)
df_final['cleaned'] = df_final.cleaned.apply(func = remove_punctuation)
df_final['cleaned'] = df_final.cleaned.apply(func = remove_html)
df_final.dropna(inplace=True)
df_final.reset_index(drop=True, inplace=True)
df_final['combine'] = df_final['combine'].apply(func = make_lower_case)
df_final['combine'] = df_final['combine'].apply(func = remove_stop_words)
df_final['combine'] = df_final['combine'].apply(func = remove_punctuation)



In [ ]:
tagged_data = [TaggedDocument(words=word_tokenize(row['combine']), tags=[row['Title']]) for index, row in df_final.iterrows()]
tagged_data


In [ ]:
print(tagged_data[471].tags, ': ', (tagged_data[471].words))


['Yüzüklerin Efendisi: İki Kule'] :  ['yüzüklerin', 'efendisi', 'i̇ki', 'kule', 'fantastik', 'macera', 'elijah', 'wood', 'sean', 'astin', 'viggo', 'mortensen', 'peter', 'jackson', 'fantastik', 'macera', 'fantastik', 'macera']


In [ ]:
model_dbow = Doc2Vec(
    dm=0, dbow_words=1,
    vector_size=200, window=8, epochs=20, workers=12,
)

model_dbow.build_vocab(tagged_data)
print(model_dbow)
    


2023-01-31 00:12:58,491 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dbow+w,d200,n5,w8,mc5,s0.001,t12>', 'datetime': '2023-01-31T00:12:58.491472', 'gensim': '4.3.0', 'python': '3.10.8 (tags/v3.10.8:aaaf517, Oct 11 2022, 16:50:30) [MSC v.1933 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}
2023-01-31 00:12:58,497 : INFO : collecting all words and their counts
2023-01-31 00:12:58,498 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2023-01-31 00:12:58,538 : INFO : PROGRESS: at example #10000, processed 164189 words (4131507 words/s), 26097 word types, 9473 tags
2023-01-31 00:12:58,813 : INFO : collected 44828 word types and 18420 unique tags from a corpus of 19433 examples and 317303 words
2023-01-31 00:12:58,813 : INFO : Creating a fresh vocabulary
2023-01-31 00:12:58,839 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=5 retains 7868 unique words (17.55% of original 44828, drops 36960)', 'datetime'

Doc2Vec<dbow+w,d200,n5,w8,mc5,s0.001,t12>


In [ ]:
model_dbow.train(tagged_data, total_examples=model_dbow.corpus_count, epochs=model_dbow.epochs)
model_dbow.save("doc2vec.model")


2023-01-31 00:12:59,869 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 12 workers on 7868 vocabulary and 200 features, using sg=1 hs=0 sample=0.001 negative=5 window=8 shrink_windows=True', 'datetime': '2023-01-31T00:12:59.869902', 'gensim': '4.3.0', 'python': '3.10.8 (tags/v3.10.8:aaaf517, Oct 11 2022, 16:50:30) [MSC v.1933 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'train'}
2023-01-31 00:13:00,928 : INFO : EPOCH 0 - PROGRESS: at 40.75% examples, 80491 words/s, in_qsize 19, out_qsize 0
2023-01-31 00:13:01,301 : INFO : EPOCH 0: training on 317303 raw words (197100 effective words) took 1.4s, 139394 effective words/s
2023-01-31 00:13:02,355 : INFO : EPOCH 1 - PROGRESS: at 40.75% examples, 80557 words/s, in_qsize 19, out_qsize 0
2023-01-31 00:13:02,725 : INFO : EPOCH 1: training on 317303 raw words (197230 effective words) took 1.4s, 139774 effective words/s
2023-01-31 00:13:03,780 : INFO : EPOCH 2 - PROGRESS: at 40.75% examples, 80269 words/s, in_

In [ ]:
def search_by_tag(tag):
    for doc in tagged_data:
        if doc.tags[0] == tag:
            return " ".join(doc.words)

def return_most_similar(mov_title) -> list:
  tokens = remove_html(remove_punctuation(remove_stop_words(make_lower_case(search_by_tag(mov_title))))).split()
  vector = model_dbow.infer_vector(tokens)
  most_similar = model_dbow.dv.most_similar([vector], topn=20) 
  return most_similar

def return_token(mov_title) -> list:
  tokens = remove_html(remove_punctuation(remove_stop_words(make_lower_case(search_by_tag(mov_title))))).split()
  return tokens

def return_most_similar_token(vec) -> list:
  # len(df_final['Title']
  most_similar = model_dbow.dv.most_similar([vec],topn=20)
  return most_similar
pp = pprint.PrettyPrinter(indent=4)
seed_text = 'Iron Man'
x = return_most_similar(seed_text)
pp.pprint(x)
print(search_by_tag(seed_text))

[   ('Iron Man', 0.9331340193748474),
    ('Iron Man 2', 0.8653346300125122),
    ('Red Tails', 0.8387789130210876),
    ('Iron Man 3', 0.828249990940094),
    ('Yüksek Gerilim', 0.7956938743591309),
    ('Yenilmezler', 0.7619659304618835),
    ('Yenilmezler: Ultron Çağı', 0.7592324614524841),
    ('Avengers: Sonsuzluk Savaşı', 0.7482845187187195),
    ('Sky Captain ve Yarının Dünyası', 0.7437907457351685),
    ('Örümcek-Adam: Eve Dönüş', 0.7393853068351746),
    ('Kaptan Amerika: Kahramanların Savaşı', 0.7380862236022949),
    ('Tuff Turf', 0.7370367050170898),
    ('Bayou Caviar', 0.7367225289344788),
    ('Sherlock Holmes 3', 0.7336269021034241),
    ('Avengers: Endgame', 0.7331240773200989),
    ('Onurlu Bir Adam', 0.7296603918075562),
    ('Bumblebee', 0.7287909388542175),
    ('Last Vegas', 0.7230497002601624),
    ('Sherlock Holmes', 0.717623233795166),
    ('Dövüş', 0.7163782715797424)]
iron man aksiyon bilimkurgu robert downey jr terrence howard gwyneth paltrow jon favreau aks

In [ ]:
seed_text = 'Dune:Çöl Gezegeni'
x = return_most_similar(seed_text)
pp.pprint(x)

[   ('Dune:Çöl Gezegeni', 0.961217999458313),
    ('Dune: Part Two', 0.9138978123664856),
    ('Star Wars: Son Jedi', 0.7451470494270325),
    ('Aşkın Algoritması', 0.7336875796318054),
    ('Zihin Gezgini', 0.7322977185249329),
    ('Hayat', 0.7309545874595642),
    ("Won't Back Down", 0.7253426909446716),
    ("Star Wars: Skywalker'ın Yükselişi", 0.7190107107162476),
    ('Minor Premise', 0.7155300974845886),
    ('Planetarium', 0.7129839658737183),
    ('Aniara', 0.710999071598053),
    ('Ex Machina', 0.7053631544113159),
    ('Geliş', 0.7024556994438171),
    ('Parallel', 0.6997082233428955),
    ('Issız Toprak', 0.697783887386322),
    ('Ocak Ayının İki Yüzü', 0.6928343772888184),
    ('Sputnik', 0.6919752955436707),
    ('Atlantis', 0.6883062720298767),
    ('Self/less', 0.6864892840385437),
    ('Trendeki Kız', 0.6845117807388306)]


In [ ]:
seed_text = 'Yüzüklerin Efendisi: İki Kule'
x = return_most_similar(seed_text)
pp.pprint(x)

[   ('Yüzüklerin Efendisi: İki Kule', 0.9510872960090637),
    ('Yüzüklerin Efendisi: Yüzük Kardeşliği', 0.8392236232757568),
    ('Gel Git Ülkesi', 0.6980528831481934),
    ('Hobbit: Beklenmedik Yolculuk', 0.6727127432823181),
    ('Krull', 0.6693352460861206),
    ('Time Bandits', 0.6654855608940125),
    ('Dr. Parnassus', 0.6651387810707092),
    ('Yeşil Rehber', 0.661405622959137),
    ('Bad Taste', 0.6503503918647766),
    ("Hobbit: Smaug'un Çorak Toprakları", 0.6499523520469666),
    ('Marionette', 0.6486194133758545),
    ('Yeşil Fener', 0.6470634341239929),
    ("Fantastik Canavarlar: Dumbledore'un Sırları", 0.6468813419342041),
    ('On Üç Yaşam', 0.6450392603874207),
    ("Pete's Dragon", 0.6345736384391785),
    ('Rumpelstiltskin', 0.6335000991821289),
    ('Yeşil Sokak Holiganları', 0.6302095055580139),
    ('İyi Evlat', 0.6301942467689514),
    ('Düşüş', 0.6293549537658691),
    ('Hidalgo', 0.6273329257965088)]


In [ ]:
seed_text = 'Rashômon'
x = return_most_similar(seed_text)
pp.pprint(x)

[   ('Rashômon', 0.9556987881660461),
    ('Yojimbo', 0.8865203261375427),
    ('Yüksek ve Alçak', 0.8857740759849548),
    ('Sanjuro', 0.8751055002212524),
    ('Akahige', 0.8685223460197449),
    ('Dodesukaden', 0.8676512837409973),
    ('Tokyo Sonatı', 0.8661526441574097),
    ('Dava Vekili', 0.8660681843757629),
    ('Kanlı Taht', 0.8626106381416321),
    ('Ran', 0.8617030382156372),
    ('Cold Fish', 0.8397113680839539),
    ('Floating Weeds', 0.8392375707626343),
    ('Quill', 0.838386595249176),
    ('Aku No Kyoten', 0.8374155163764954),
    ('Cut', 0.8371406197547913),
    ('Chikamatsu monogatari', 0.8339501023292542),
    ('Seppuku', 0.8263356685638428),
    ('Yedi Samuray', 0.8261293768882751),
    ('Ichimei', 0.8261224031448364),
    ('Gizli Kale', 0.8252803683280945)]


In [ ]:
seed_text = 'Titanik'
x = return_most_similar(seed_text)
pp.pprint(x)

[   ('Titanik', 0.9392005801200867),
    ('Boşanma', 0.7207993865013123),
    ('Tatil', 0.7026091814041138),
    ('Hayallerin Peşinde', 0.6995029449462891),
    ("Don's Plum", 0.6978124976158142),
    ('Aramızdaki Sözler', 0.686307966709137),
    ('Gece ve Gündüz', 0.6801105737686157),
    ('Bu Dünya ve Ötesi', 0.6797755360603333),
    ('Şöhrete İlk Adım', 0.6792684197425842),
    ('Cowboy Bob', 0.6785457134246826),
    ('Kutsal Duman', 0.6759546995162964),
    ('Elizabethtown', 0.6757436394691467),
    ('Enigma', 0.6734517812728882),
    ('Straw Dogs', 0.6662154197692871),
    ('Blackbird', 0.6641435623168945),
    ('Orlando', 0.6640946865081787),
    ('Tutku Oyunları', 0.6604956388473511),
    ('Amsterdam Ekspres', 0.6599982380867004),
    ('Wonderland', 0.6597193479537964),
    ('Bir Tutam Cennet', 0.6568059921264648)]


In [ ]:
# this dict has title:embedding -> embeddings are created using that specific movie's plot
movie_embeddings = {title: model_dbow.infer_vector(return_token(title)) for title in df_final['Title']}

with open("mov_embeddings_other.pickle", "wb") as f:
    pickle.dump(movie_embeddings, f)

In [ ]:
with open('mov_embeddings_desc.pickle', 'rb') as f:
    desc = pickle.load(f)
with open('mov_embeddings_other.pickle', 'rb') as f_1:
    other = pickle.load(f_1)

In [ ]:
def merge_dicts(dict1, dict2):
    merged_dict = {}
    for title, embedding1 in dict1.items():
        embedding2 = dict2[title]
        merged_embedding = (embedding1 * 0.2 + embedding2 * 0.8)
        merged_dict[title] = merged_embedding
    return merged_dict
merged_dict = merge_dicts(desc, other)


In [ ]:
mov_title = 'Hababam Sınıfı Uyanıyor'
closest_movies = return_most_similar_token(merged_dict[mov_title])
closest_movies

[('Hababam Sınıfı Sınıfta Kaldı', 0.8722280263900757),
 ('Hababam Sınıfı Uyanıyor', 0.8670487403869629),
 ('Hababam Sınıfı Güle Güle', 0.8633586168289185),
 ('Sakar Şakir', 0.8589163422584534),
 ('Hababam Sınıfı Tatilde', 0.8577638864517212),
 ('Banker Bilo', 0.8551812171936035),
 ('Köyden İndim Şehire', 0.8541895747184753),
 ('Erkek Güzeli Sefil Bilo', 0.853534996509552),
 ('Şen Dul Şaban', 0.8531508445739746),
 ('Salak Milyoner', 0.85310298204422),
 ('Şabanoğlu Şaban', 0.8498935103416443),
 ('Hababam Sınıfı', 0.8483697772026062),
 ('Namuslu', 0.8469749093055725),
 ('Küçük Hanımefendi', 0.8452606201171875),
 ('Yalancı Yarim', 0.8446537256240845),
 ('Hababam Sınıfı Merhaba', 0.8427131772041321),
 ('Hanzo', 0.8419997692108154),
 ('Milyarder', 0.8419749736785889),
 ('Çarıklı Milyoner', 0.8408260345458984),
 ('Davaro', 0.8399199843406677)]

In [ ]:
with open("mov_embeddings_averaged.pickle", "wb") as f:
    pickle.dump(merged_dict, f)

#### Reading from pickle


In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import gensim
import nltk
from scipy.spatial import distance
import re
from random import shuffle
import string
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec    
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn import utils
from sklearn.model_selection import train_test_split
import ast
import pickle
import pprint
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
with open('mov_embeddings_averaged.pickle', 'rb') as f_1:
    embeds = pickle.load(f_1)
    
def return_rank_list(embeds:dict, mov_title:str) -> list:

  selected_vector = embeds[mov_title]

  similarities = []
  for movie_title, vector in embeds.items():
      cos_sim = 1 - distance.cosine(selected_vector, vector)
      similarities.append((movie_title, cos_sim))

  ranked_movies = sorted(similarities, key=lambda x: x[1], reverse=True)
  ranked_movies = [x for x in ranked_movies if x[0] != mov_title]


  dct = {}
  # replacing cosine similarities with ranks
  for index, item in enumerate(ranked_movies):
    itemlist = list(item)
    # itemlist[1] = index + 1
    # item = tuple(itemlist)

    # ranked_movies[index] = item

    dct[itemlist[0]] = index+1


  return dct


In [3]:
with open('mov_embeds_avg_new.pickle', 'wb') as f_1:
    pickle.dump(embeds, f_1, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
# guess_movie = 'Iron Man'
# valid_movies = list(embeds.keys())
# game = True
# sim_movies_list = return_rank_list(embeds,guess_movie)
# while game:
#     user_input = input()
#     if user_input not in valid_movies:
#         print('I do not know this movie')
#     else:
#         if (user_input == guess_movie):
#             print('You have won.')
#             break
#         print(f'{user_input}:{sim_movies_list[user_input]}')



In [5]:
import customtkinter
import tkinter as tk
guess_movie = "Hababam Sınıfı Uyanıyor"
valid_movies = list(embeds.keys())
sim_movies_list = return_rank_list(embeds,guess_movie)


In [8]:
def main_window():
    customtkinter.set_appearance_mode("dark")
    customtkinter.set_default_color_theme("dark-blue")

    root = customtkinter.CTk()
    root.title("Movie Guess Game")
    root.geometry("650x450")
    frame = customtkinter.CTkFrame(master=root)
    frame.pack(pady=15, padx=30, fill="both",expand=True)

    label = customtkinter.CTkLabel(frame, text="Guess the movie", font=("Helvetica", 16))
    label.pack(pady=15, padx=30, fill="both",expand=True)

    guess_entry = customtkinter.CTkEntry(frame, font=("Helvetica", 14))
    guess_entry.pack(pady=10)

    previous_guesses = []
    def game_check(user_input):
        if (user_input == guess_movie):
            result_label.configure(text="You have won. You may close the window.")
            guess_entry.configure(state="disabled")
            guess_button.configure(state="disabled")

    def check_guess():
        user_input = guess_entry.get().strip() 
        game_check(user_input)    
        if user_input not in valid_movies:
            result_label.configure(text="I do not know this movie")
        else:
            game_check(user_input)
            previous_guesses.append((user_input, sim_movies_list[user_input]))
            previous_guesses.sort(key=lambda x: x[1], reverse=False)
            result_label.configure(text=f"{user_input}:{sim_movies_list[user_input]}")
        previous_guesses_list.delete(0, tk.END)
        for guess in previous_guesses:
            previous_guesses_list.insert(tk.END, f"{guess[0]}:{guess[1]}")

    guess_button = customtkinter.CTkButton(frame, text="Guess", command=check_guess, font=("Helvetica", 14))
    guess_button.pack(pady=10)

    result_label = customtkinter.CTkLabel(master=frame, text=f"")
    result_label.pack(pady=20)

    previous_guesses_label = customtkinter.CTkLabel(frame, text="Previous guesses:", font=("Helvetica", 14))
    previous_guesses_label.pack(pady=10)


    previous_guesses_list = tk.Listbox(frame, fg = "white" ,bg = "#262629", font=("Helvetica", 14))
    previous_guesses_list.pack(pady=10)

    root.mainloop()

main_window()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\customtkinter\windows\widgets\ctk_button.py", line 553, in _clicked
    self._command()
  File "C:\Users\User\AppData\Local\Temp\ipykernel_11340\45447972.py", line 31, in check_guess
    previous_guesses.append((user_input, sim_movies_list[user_input]))
KeyError: 'Recep İvedik 5'


In [6]:
guess_movie = "Recep İvedik 5"
valid_movies = list(embeds.keys())
sim_movies_list = return_rank_list(embeds,guess_movie)
sim_movies_list

{'Recep İvedik 3': 1,
 'Recep İvedik 4': 2,
 'Recep İvedik': 3,
 'Recep İvedik 7': 4,
 'Recep İvedik 2': 5,
 'Osman Pazarlama': 6,
 'Celal ile Ceren': 7,
 'Recep İvedik 6': 8,
 'Kuru Otlar Üstüne': 9,
 'Feride': 10,
 'Yalancı Şahit': 11,
 'Bulutların Üstünde': 12,
 'Bayram Şekeri': 13,
 'Kayhan': 14,
 'Dedemin Fişi': 15,
 'Hep Yek 5: Bizim Şeyimiz mi Altan': 16,
 'Avanak Dedektör': 17,
 'Güvercin Uçuverdi': 18,
 'Hep Yek 2': 19,
 '41 Kere Maşallah': 20,
 'Posta Kutusu': 21,
 "Abbas'ın Melekleri": 22,
 '7 Melek': 23,
 'Hep Yek 4: Bela Okuma Altan': 24,
 'Leblebi Tozu': 25,
 'Öt Bakalım': 26,
 'Ölü Yatırım': 27,
 'Hep Yek 3': 28,
 'Tilki Yuvası': 29,
 'Kaygı': 30,
 'Zengo': 31,
 'Kayseri Aslanı: Çin İşi': 32,
 'Eyvah Karım': 33,
 'Deliha 2': 34,
 'Figüran': 35,
 'Baba Mirası': 36,
 'Çam Yarması 2': 37,
 'Arınma Seansı - Tarot': 38,
 'Sağ Salim': 39,
 'Aşkın Tadı': 40,
 'Muhalif Başkan': 41,
 'Kafkas': 42,
 'Deli Dumrul': 43,
 'Vezir Parmağı': 44,
 'Ulan Salih': 45,
 'Gülcemal': 46,
 'Boğ

In [ ]:
def main_window():
    root = tk.Tk()
    root.title("Movie Guess Game")
    root.geometry("650x450")
    root.config(bg='#303030')

    label = tk.Label(root, text="Guess the movie", bg='#303030', fg='white', font=("Helvetica", 16))
    label.pack(pady=20)

    guess_entry = tk.Entry(root, bg='white', fg='#303030', font=("Helvetica", 14))
    guess_entry.pack(pady=10)

    previous_guesses = []

    def check_guess():
        
        user_input = guess_entry.get()
        if (user_input == guess_movie):
            result_label.config(text="You have won.", bg='#303030', fg='white')
            guess_entry.config(state="disabled")
            guess_button.config(state="disabled")
        if user_input not in valid_movies:
            result_label.config(text="I do not know this movie", bg='#303030', fg='white')
        else:
            previous_guesses.append((user_input, sim_movies_list[user_input]))
            previous_guesses.sort(key=lambda x: x[1], reverse=False)
            if (user_input == guess_movie):
                result_label.config(text="You have won.", bg='#303030', fg='white')
                guess_entry.config(state="disabled")
                guess_button.config(state="disabled")
        previous_guesses_list.delete(0, tk.END)
        for guess in previous_guesses:
            previous_guesses_list.insert(tk.END, f"{guess[0]}:{guess[1]}")

    guess_button = tk.Button(root, text="Guess", command=check_guess, bg='#303030', fg='white', font=("Helvetica", 14))
    guess_button.pack(pady=10)

    result_label = tk.Label(root, bg='#303030', fg='white')
    result_label.pack(pady=20)

    previous_guesses_label = tk.Label(root, text="Previous guesses:", bg='#303030', fg='white', font=("Helvetica", 14))
    previous_guesses_label.pack(pady=10)

    previous_guesses_list = tk.Listbox(root, bg='white', fg='#303030', font=("Helvetica", 14))
    previous_guesses_list.pack(pady=10)

    root.mainloop()

main_window()